# Criptografía: Conceptos y Aplicaciones Prácticas con Python

## Introducción

La criptografía es la ciencia de proteger la información mediante la transformación de datos en formatos que no son fácilmente comprensibles. En este notebook, exploraremos los conceptos fundamentales de la criptografía y su implementación práctica utilizando Python.

### Temas que cubriremos:
1. Cifrado simétrico
2. Cifrado asimétrico
3. Funciones hash
4. Firmas digitales
5. Aplicaciones prácticas


## Preparación del entorno

Primero, instalemos las bibliotecas necesarias para nuestros ejemplos:


In [1]:
# Instalación de bibliotecas necesarias
# Descomenta estas líneas si necesitas instalar las bibliotecas
# !pip install pycryptodome
# !pip install cryptography

In [3]:
# Importamos las bibliotecas que utilizaremos
import os
import hashlib
import base64
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC


## 1. Cifrado Simétrico

El cifrado simétrico utiliza la misma clave para cifrar y descifrar datos. Es rápido y eficiente para grandes volúmenes de datos.

### 1.1 Cifrado César (método clásico)

El cifrado César es uno de los métodos de cifrado más simples y antiguos. Consiste en desplazar cada letra del mensaje un número fijo de posiciones en el alfabeto.


In [4]:
def cifrado_cesar(texto, desplazamiento):
    """
    Implementación del cifrado César.
    
    Args:
        texto: El texto a cifrar
        desplazamiento: El número de posiciones a desplazar cada letra
        
    Returns:
        El texto cifrado
    """
    resultado = ""
    
    for char in texto:
        if char.isalpha():
            ascii_offset = ord('a') if char.islower() else ord('A')
            # Fórmula: (posición_original + desplazamiento) % 26
            resultado += chr((ord(char) - ascii_offset + desplazamiento) % 26 + ascii_offset)
        else:
            resultado += char
            
    return resultado

def descifrado_cesar(texto_cifrado, desplazamiento):
    """
    Descifra un texto cifrado con el método César.
    
    Args:
        texto_cifrado: El texto cifrado
        desplazamiento: El desplazamiento utilizado para cifrar
        
    Returns:
        El texto original
    """
    return cifrado_cesar(texto_cifrado, -desplazamiento)

# Ejemplo de uso
mensaje_original = "Hola Mundo"
desplazamiento = 3

mensaje_cifrado = cifrado_cesar(mensaje_original, desplazamiento)
mensaje_descifrado = descifrado_cesar(mensaje_cifrado, desplazamiento)

print(f"Mensaje original: {mensaje_original}")
print(f"Mensaje cifrado: {mensaje_cifrado}")
print(f"Mensaje descifrado: {mensaje_descifrado}")


Mensaje original: Hola Mundo
Mensaje cifrado: Krod Pxqgr
Mensaje descifrado: Hola Mundo


### 1.2 Cifrado AES (Advanced Encryption Standard)

AES es uno de los algoritmos de cifrado simétrico más utilizados en la actualidad. Es seguro y eficiente, y se utiliza en muchas aplicaciones y protocolos de seguridad.


In [5]:
def generar_clave_aes():
    """
    Genera una clave aleatoria para AES-256.
    
    Returns:
        Una clave de 32 bytes (256 bits)
    """
    return os.urandom(32)  # 32 bytes = 256 bits

def cifrar_aes(mensaje, clave):
    """
    Cifra un mensaje utilizando AES en modo GCM.
    
    Args:
        mensaje: El mensaje a cifrar (bytes)
        clave: La clave de cifrado (32 bytes)
        
    Returns:
        Una tupla (nonce, texto_cifrado, tag)
    """
    # Generamos un nonce aleatorio
    nonce = os.urandom(12)
    
    # Creamos el cifrador
    cifrador = Cipher(algorithms.AES(clave), modes.GCM(nonce))
    encriptador = cifrador.encryptor()
    
    # Ciframos el mensaje
    texto_cifrado = encriptador.update(mensaje) + encriptador.finalize()
    
    return (nonce, texto_cifrado, encriptador.tag)

def descifrar_aes(nonce, texto_cifrado, tag, clave):
    """
    Descifra un mensaje cifrado con AES en modo GCM.
    
    Args:
        nonce: El nonce utilizado para cifrar
        texto_cifrado: El texto cifrado
        tag: El tag de autenticación
        clave: La clave de cifrado
        
    Returns:
        El mensaje original
    """
    # Creamos el descifrador
    descifrador = Cipher(algorithms.AES(clave), modes.GCM(nonce, tag)).decryptor()
    
    # Desciframos el mensaje
    return descifrador.update(texto_cifrado) + descifrador.finalize()

# Ejemplo de uso
mensaje = b"Este es un mensaje secreto que quiero cifrar con AES."
clave = generar_clave_aes()

# Ciframos el mensaje
nonce, texto_cifrado, tag = cifrar_aes(mensaje, clave)

# Desciframos el mensaje
mensaje_descifrado = descifrar_aes(nonce, texto_cifrado, tag, clave)

print(f"Mensaje original: {mensaje.decode()}")
print(f"Mensaje cifrado (hex): {texto_cifrado.hex()}")
print(f"Mensaje descifrado: {mensaje_descifrado.decode()}")


Mensaje original: Este es un mensaje secreto que quiero cifrar con AES.
Mensaje cifrado (hex): 1c81371963b06c30fd976c4656a9f86c594887ff8f9a443ceb7fbcaa6730585b00cae183b992ddc3e4bf0908bda58a68db39760b0d
Mensaje descifrado: Este es un mensaje secreto que quiero cifrar con AES.


### 1.3 Derivación de claves a partir de contraseñas

En la práctica, a menudo necesitamos generar claves criptográficas a partir de contraseñas. Para esto, utilizamos funciones de derivación de claves como PBKDF2.


In [6]:
def derivar_clave_de_password(password, salt=None):
    """
    Deriva una clave criptográfica a partir de una contraseña utilizando PBKDF2.
    
    Args:
        password: La contraseña (string)
        salt: La sal a utilizar (generada aleatoriamente si no se proporciona)
        
    Returns:
        Una tupla (clave, salt)
    """
    if salt is None:
        salt = os.urandom(16)
        
    # Convertimos la contraseña a bytes si es necesario
    if isinstance(password, str):
        password = password.encode()
        
    # Configuramos PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 32 bytes = 256 bits
        salt=salt,
        iterations=100000,  # Número de iteraciones (mayor = más seguro pero más lento)
    )
    
    # Derivamos la clave
    clave = kdf.derive(password)
    
    return clave, salt

# Ejemplo de uso
password = "mi_contraseña_secreta"

# Derivamos una clave a partir de la contraseña
clave, salt = derivar_clave_de_password(password)

print(f"Contraseña: {password}")
print(f"Sal (hex): {salt.hex()}")
print(f"Clave derivada (hex): {clave.hex()}")

# Verificamos que podemos regenerar la misma clave con la misma contraseña y sal
clave2, _ = derivar_clave_de_password(password, salt)
print(f"¿Las claves coinciden? {clave == clave2}")


Contraseña: mi_contraseña_secreta
Sal (hex): ed8d2a56584a633786fb65dbaf5a68b0
Clave derivada (hex): 5d7d7dd6a480248c5a788e8a5ff8d00b7f73f3b8286c38ec9da8a97b09d38942
¿Las claves coinciden? True


## 2. Cifrado Asimétrico

El cifrado asimétrico utiliza un par de claves: una pública para cifrar y una privada para descifrar. Esto resuelve el problema de intercambio de claves del cifrado simétrico.

### 2.1 RSA (Rivest-Shamir-Adleman)

RSA es uno de los algoritmos de cifrado asimétrico más utilizados. Se basa en la dificultad de factorizar números grandes.


In [7]:
def generar_par_claves_rsa(tamano_bits=2048):
    """
    Genera un par de claves RSA.
    
    Args:
        tamano_bits: El tamaño de la clave en bits
        
    Returns:
        Una tupla (clave_privada, clave_publica)
    """
    # Generamos la clave privada
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,  # Exponente público estándar
        key_size=tamano_bits
    )
    
    # Obtenemos la clave pública correspondiente
    clave_publica = clave_privada.public_key()
    
    return clave_privada, clave_publica

def cifrar_rsa(mensaje, clave_publica):
    """
    Cifra un mensaje utilizando RSA.
    
    Args:
        mensaje: El mensaje a cifrar (bytes)
        clave_publica: La clave pública RSA
        
    Returns:
        El mensaje cifrado
    """
    texto_cifrado = clave_publica.encrypt(
        mensaje,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return texto_cifrado

def descifrar_rsa(texto_cifrado, clave_privada):
    """
    Descifra un mensaje cifrado con RSA.
    
    Args:
        texto_cifrado: El texto cifrado
        clave_privada: La clave privada RSA
        
    Returns:
        El mensaje original
    """
    mensaje_original = clave_privada.decrypt(
        texto_cifrado,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_original

# Ejemplo de uso
# Nota: RSA tiene limitaciones en el tamaño del mensaje que puede cifrar directamente
mensaje = b"Mensaje secreto para cifrar con RSA"

# Generamos un par de claves
clave_privada, clave_publica = generar_par_claves_rsa()

# Ciframos el mensaje con la clave pública
texto_cifrado = cifrar_rsa(mensaje, clave_publica)

# Desciframos el mensaje con la clave privada
mensaje_descifrado = descifrar_rsa(texto_cifrado, clave_privada)

print(f"Mensaje original: {mensaje.decode()}")
print(f"Mensaje cifrado (primeros 50 bytes en hex): {texto_cifrado[:50].hex()}...")
print(f"Mensaje descifrado: {mensaje_descifrado.decode()}")


Mensaje original: Mensaje secreto para cifrar con RSA
Mensaje cifrado (primeros 50 bytes en hex): 4a03fd41d952a98d4e7e0a2699c2ae37884dd22aa3eb621c0fd1ea089f53878948ca4d151dfbe321a4f12e85de11f1d13564...
Mensaje descifrado: Mensaje secreto para cifrar con RSA


### 2.2 Cifrado híbrido

En la práctica, a menudo se utiliza un enfoque híbrido: se cifra una clave simétrica con RSA, y luego se utiliza esa clave simétrica para cifrar el mensaje real con AES. Esto combina la seguridad del cifrado asimétrico con la eficiencia del cifrado simétrico.


In [10]:
def cifrado_hibrido(mensaje, clave_publica):
    """
    Implementa un cifrado híbrido: genera una clave AES, cifra el mensaje con AES,
    y luego cifra la clave AES con RSA.
    
    Args:
        mensaje: El mensaje a cifrar (bytes)
        clave_publica: La clave pública RSA
        
    Returns:
        Una tupla (clave_aes_cifrada, nonce, texto_cifrado, tag)
    """
    # Generamos una clave AES aleatoria
    clave_aes = os.urandom(32)
    
    # Ciframos el mensaje con AES
    nonce, texto_cifrado, tag = cifrar_aes(mensaje, clave_aes)
    
    # Ciframos la clave AES con RSA
    clave_aes_cifrada = cifrar_rsa(clave_aes, clave_publica)
    
    return (clave_aes_cifrada, nonce, texto_cifrado, tag)

def descifrado_hibrido(clave_aes_cifrada, nonce, texto_cifrado, tag, clave_privada):
    """
    Descifra un mensaje cifrado con el método híbrido.
    
    Args:
        clave_aes_cifrada: La clave AES cifrada con RSA
        nonce: El nonce utilizado para AES
        texto_cifrado: El texto cifrado con AES
        tag: El tag de autenticación de AES-GCM
        clave_privada: La clave privada RSA
        
    Returns:
        El mensaje original
    """
    # Desciframos la clave AES con RSA
    clave_aes = descifrar_rsa(clave_aes_cifrada, clave_privada)
    
    # Desciframos el mensaje con AES
    mensaje_original = descifrar_aes(nonce, texto_cifrado, tag, clave_aes)
    
    return mensaje_original

# Ejemplo de uso
mensaje = b"Este es un mensaje muy largo que queremos cifrar de manera segura utilizando un enfoque hibrido que combina la seguridad de RSA con la eficiencia de AES."

# Generamos un par de claves RSA
clave_privada, clave_publica = generar_par_claves_rsa()

# Ciframos el mensaje con el método híbrido
clave_aes_cifrada, nonce, texto_cifrado, tag = cifrado_hibrido(mensaje, clave_publica)

# Desciframos el mensaje
mensaje_descifrado = descifrado_hibrido(clave_aes_cifrada, nonce, texto_cifrado, tag, clave_privada)

print(f"Mensaje original: {mensaje.decode()}")
print(f"Mensaje descifrado: {mensaje_descifrado.decode()}")


Mensaje original: Este es un mensaje muy largo que queremos cifrar de manera segura utilizando un enfoque hibrido que combina la seguridad de RSA con la eficiencia de AES.
Mensaje descifrado: Este es un mensaje muy largo que queremos cifrar de manera segura utilizando un enfoque hibrido que combina la seguridad de RSA con la eficiencia de AES.


## 3. Funciones Hash

Las funciones hash convierten datos de cualquier tamaño en una cadena de longitud fija. Son útiles para verificar la integridad de los datos y almacenar contraseñas de manera segura.

### 3.1 Funciones hash criptográficas (SHA-256, SHA-512)


In [11]:
def calcular_hash(datos, algoritmo='sha256'):
    """
    Calcula el hash de los datos utilizando el algoritmo especificado.
    
    Args:
        datos: Los datos a hashear (bytes o string)
        algoritmo: El algoritmo de hash a utilizar ('sha256', 'sha512', 'md5', etc.)
        
    Returns:
        El hash en formato hexadecimal
    """
    # Convertimos a bytes si es necesario
    if isinstance(datos, str):
        datos = datos.encode()
        
    # Creamos el objeto hash
    h = hashlib.new(algoritmo)
    
    # Actualizamos con los datos
    h.update(datos)
    
    # Devolvemos el hash en formato hexadecimal
    return h.hexdigest()

# Ejemplos de uso
mensaje = "Hola Mundo"

print(f"Mensaje: {mensaje}")
print(f"SHA-256: {calcular_hash(mensaje, 'sha256')}")
print(f"SHA-512: {calcular_hash(mensaje, 'sha512')}")
print(f"MD5: {calcular_hash(mensaje, 'md5')}")

# Demostración de la propiedad de avalancha
mensaje2 = "Hola Mundd"  # Cambiamos una sola letra
print(f"\nMensaje modificado: {mensaje2}")
print(f"SHA-256: {calcular_hash(mensaje2, 'sha256')}")


Mensaje: Hola Mundo
SHA-256: c3a4a2e49d91f2177113a9adfcb9ef9af9679dc4557a0a3a4602e1bd39a6f481
SHA-512: bb0b2687b9f11a75110cbf78570ea7c8babb10880d533a1ab388113871bd643c47db96e1b19793476b5b3f3000071961aa6d6cab0c1c873c4d7515512c62bbbb
MD5: d501194c987486789bb01b50dc1a0adb

Mensaje modificado: Hola Mundd
SHA-256: 690d89d9bfbb14a9538789a3a23a3300a59e3a7a3d84f1c2516be610eb95f321


### 3.2 Almacenamiento seguro de contraseñas

Nunca debemos almacenar contraseñas en texto plano. En su lugar, utilizamos funciones hash con sal para almacenarlas de manera segura.


In [13]:
def hash_password(password):
    """
    Genera un hash seguro para una contraseña utilizando PBKDF2.
    
    Args:
        password: La contraseña a hashear
        
    Returns:
        Una cadena que contiene la sal y el hash, separados por '$'
    """
    # Generamos una sal aleatoria
    salt = os.urandom(16)
    
    # Convertimos la contraseña a bytes si es necesario
    if isinstance(password, str):
        password = password.encode()
    
    # Utilizamos PBKDF2 para derivar una clave
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
    )
    hash_password = kdf.derive(password)
    
    # Codificamos la sal y el hash en base64 para almacenamiento
    salt_b64 = base64.b64encode(salt).decode()
    hash_b64 = base64.b64encode(hash_password).decode()
    
    # Devolvemos la sal y el hash separados por '$'
    return f"{salt_b64}${hash_b64}"

def verificar_password(password, hash_almacenado):
    """
    Verifica si una contraseña coincide con un hash almacenado.
    
    Args:
        password: La contraseña a verificar
        hash_almacenado: El hash almacenado (sal + hash)
        
    Returns:
        True si la contraseña coincide, False en caso contrario
    """
    # Separamos la sal y el hash
    salt_b64, hash_b64 = hash_almacenado.split('$')
    
    # Decodificamos la sal y el hash
    salt = base64.b64decode(salt_b64)
    hash_almacenado_bytes = base64.b64decode(hash_b64)
    
    # Convertimos la contraseña a bytes si es necesario
    if isinstance(password, str):
        password = password.encode()
    
    # Utilizamos PBKDF2 para derivar una clave con la misma sal
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
    )
    
    try:
        # Verificamos si el hash coincide
        kdf.verify(password, hash_almacenado_bytes)
        return True
    except Exception:
        return False

# Ejemplo de uso
password = "mi_contraseña_secreta"

# Hasheamos la contraseña para almacenarla
hash_almacenado = hash_password(password)
print(f"Contraseña: {password}")
print(f"Hash almacenado: {hash_almacenado}")

# Verificamos la contraseña
print(f"\nVerificación de contraseñas:")
print(f"Contraseña correcta: {verificar_password(password, hash_almacenado)}")
print(f"Contraseña incorrecta: {verificar_password('contraseña_incorrecta', hash_almacenado)}")


Contraseña: mi_contraseña_secreta
Hash almacenado: I7hXY2hW3LaahT5EXH16LQ==$rY4WuL4a/E0797rIO6WdLRD9F3n75M4tvtyH/FoVJHg=

Verificación de contraseñas:
Contraseña correcta: True
Contraseña incorrecta: False


## 4. Firmas Digitales

Las firmas digitales permiten verificar la autenticidad e integridad de un mensaje. Utilizan criptografía asimétrica: el remitente firma con su clave privada, y el receptor verifica con la clave pública del remitente.


In [14]:
def firmar_mensaje(mensaje, clave_privada):
    """
    Firma un mensaje utilizando RSA.
    
    Args:
        mensaje: El mensaje a firmar (bytes o string)
        clave_privada: La clave privada RSA
        
    Returns:
        La firma digital
    """
    # Convertimos a bytes si es necesario
    if isinstance(mensaje, str):
        mensaje = mensaje.encode()
    
    # Firmamos el mensaje
    firma = clave_privada.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    return firma

def verificar_firma(mensaje, firma, clave_publica):
    """
    Verifica una firma digital.
    
    Args:
        mensaje: El mensaje original (bytes o string)
        firma: La firma digital
        clave_publica: La clave pública RSA del firmante
        
    Returns:
        True si la firma es válida, False en caso contrario
    """
    # Convertimos a bytes si es necesario
    if isinstance(mensaje, str):
        mensaje = mensaje.encode()
    
    try:
        # Verificamos la firma
        clave_publica.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

# Ejemplo de uso
mensaje = "Este mensaje ha sido firmado digitalmente para garantizar su autenticidad."

# Generamos un par de claves RSA
clave_privada, clave_publica = generar_par_claves_rsa()

# Firmamos el mensaje
firma = firmar_mensaje(mensaje, clave_privada)

# Verificamos la firma
es_valida = verificar_firma(mensaje, firma, clave_publica)
print(f"Mensaje: {mensaje}")
print(f"Firma (primeros 50 bytes en hex): {firma[:50].hex()}...")
print(f"¿La firma es válida? {es_valida}")

# Verificamos con un mensaje modificado
mensaje_modificado = mensaje + " Texto añadido maliciosamente."
es_valida_modificado = verificar_firma(mensaje_modificado, firma, clave_publica)
print(f"\nMensaje modificado: {mensaje_modificado}")
print(f"¿La firma es válida para el mensaje modificado? {es_valida_modificado}")


Mensaje: Este mensaje ha sido firmado digitalmente para garantizar su autenticidad.
Firma (primeros 50 bytes en hex): 3a72b885541eda4ace344a31b231d73537b9fc59f2a82c9a6a31206a0e3b861a863c0ec50f122b8a1ddfce7c185042a6b5ee...
¿La firma es válida? True

Mensaje modificado: Este mensaje ha sido firmado digitalmente para garantizar su autenticidad. Texto añadido maliciosamente.
¿La firma es válida para el mensaje modificado? False


## 5. Aplicaciones Prácticas

### 5.1 Cifrado de archivos

Veamos cómo podemos cifrar y descifrar archivos utilizando AES.


In [15]:
def cifrar_archivo(ruta_archivo, ruta_salida, password):
    """
    Cifra un archivo utilizando AES con una clave derivada de una contraseña.
    
    Args:
        ruta_archivo: Ruta al archivo a cifrar
        ruta_salida: Ruta donde guardar el archivo cifrado
        password: Contraseña para cifrar
    """
    # Derivamos una clave a partir de la contraseña
    clave, salt = derivar_clave_de_password(password)
    
    # Leemos el archivo
    with open(ruta_archivo, 'rb') as f:
        datos = f.read()
    
    # Ciframos los datos
    nonce, datos_cifrados, tag = cifrar_aes(datos, clave)
    
    # Guardamos la sal, el nonce, el tag y los datos cifrados
    with open(ruta_salida, 'wb') as f:
        f.write(salt)  # 16 bytes
        f.write(nonce)  # 12 bytes
        f.write(tag)  # 16 bytes
        f.write(datos_cifrados)

def descifrar_archivo(ruta_archivo_cifrado, ruta_salida, password):
    """
    Descifra un archivo cifrado con AES.
    
    Args:
        ruta_archivo_cifrado: Ruta al archivo cifrado
        ruta_salida: Ruta donde guardar el archivo descifrado
        password: Contraseña para descifrar
    """
    # Leemos el archivo cifrado
    with open(ruta_archivo_cifrado, 'rb') as f:
        salt = f.read(16)
        nonce = f.read(12)
        tag = f.read(16)
        datos_cifrados = f.read()
    
    # Derivamos la clave a partir de la contraseña y la sal
    clave, _ = derivar_clave_de_password(password, salt)
    
    # Desciframos los datos
    datos_descifrados = descifrar_aes(nonce, datos_cifrados, tag, clave)
    
    # Guardamos los datos descifrados
    with open(ruta_salida, 'wb') as f:
        f.write(datos_descifrados)

# Ejemplo de uso (comentado para evitar crear archivos reales)
'''
# Creamos un archivo de ejemplo
with open('archivo_ejemplo.txt', 'w') as f:
    f.write("Este es un archivo de ejemplo que vamos a cifrar.")

# Ciframos el archivo
cifrar_archivo('archivo_ejemplo.txt', 'archivo_cifrado.bin', 'mi_contraseña')

# Desciframos el archivo
descifrar_archivo('archivo_cifrado.bin', 'archivo_descifrado.txt', 'mi_contraseña')
'''


'\n# Creamos un archivo de ejemplo\nwith open(\'archivo_ejemplo.txt\', \'w\') as f:\n    f.write("Este es un archivo de ejemplo que vamos a cifrar.")\n\n# Ciframos el archivo\ncifrar_archivo(\'archivo_ejemplo.txt\', \'archivo_cifrado.bin\', \'mi_contraseña\')\n\n# Desciframos el archivo\ndescifrar_archivo(\'archivo_cifrado.bin\', \'archivo_descifrado.txt\', \'mi_contraseña\')\n'

### 5.2 Comunicación segura

Simulemos un intercambio de mensajes seguro entre Alice y Bob utilizando cifrado híbrido y firmas digitales.


In [16]:
# Generamos pares de claves para Alice y Bob
clave_privada_alice, clave_publica_alice = generar_par_claves_rsa()
clave_privada_bob, clave_publica_bob = generar_par_claves_rsa()

def enviar_mensaje_seguro(mensaje, clave_privada_remitente, clave_publica_destinatario):
    """
    Simula el envío de un mensaje seguro: lo firma y lo cifra.
    
    Args:
        mensaje: El mensaje a enviar
        clave_privada_remitente: La clave privada del remitente (para firmar)
        clave_publica_destinatario: La clave pública del destinatario (para cifrar)
        
    Returns:
        Una tupla con todos los datos necesarios para recibir el mensaje
    """
    # Convertimos a bytes si es necesario
    if isinstance(mensaje, str):
        mensaje = mensaje.encode()
    
    # Firmamos el mensaje
    firma = firmar_mensaje(mensaje, clave_privada_remitente)
    
    # Combinamos el mensaje y la firma
    mensaje_con_firma = mensaje + b"||FIRMA||" + firma
    
    # Ciframos el mensaje con firma utilizando cifrado híbrido
    clave_aes_cifrada, nonce, texto_cifrado, tag = cifrado_hibrido(mensaje_con_firma, clave_publica_destinatario)
    
    return (clave_aes_cifrada, nonce, texto_cifrado, tag)

def recibir_mensaje_seguro(datos_cifrados, clave_privada_destinatario, clave_publica_remitente):
    """
    Simula la recepción de un mensaje seguro: lo descifra y verifica la firma.
    
    Args:
        datos_cifrados: Los datos cifrados (resultado de enviar_mensaje_seguro)
        clave_privada_destinatario: La clave privada del destinatario (para descifrar)
        clave_publica_remitente: La clave pública del remitente (para verificar la firma)
        
    Returns:
        Una tupla (mensaje, firma_valida)
    """
    # Desempaquetamos los datos cifrados
    clave_aes_cifrada, nonce, texto_cifrado, tag = datos_cifrados
    
    # Desciframos el mensaje con firma
    mensaje_con_firma = descifrado_hibrido(clave_aes_cifrada, nonce, texto_cifrado, tag, clave_privada_destinatario)
    
    # Separamos el mensaje y la firma
    mensaje, firma = mensaje_con_firma.split(b"||FIRMA||")
    
    # Verificamos la firma
    firma_valida = verificar_firma(mensaje, firma, clave_publica_remitente)
    
    return (mensaje, firma_valida)

# Ejemplo de uso: Alice envía un mensaje a Bob
mensaje_alice = "Hola Bob, este es un mensaje secreto de Alice."
print(f"Alice quiere enviar: {mensaje_alice}")

# Alice firma y cifra el mensaje para Bob
datos_cifrados = enviar_mensaje_seguro(mensaje_alice, clave_privada_alice, clave_publica_bob)

# Bob recibe y descifra el mensaje de Alice
mensaje_recibido, firma_valida = recibir_mensaje_seguro(datos_cifrados, clave_privada_bob, clave_publica_alice)

print(f"Bob recibe: {mensaje_recibido.decode()}")
print(f"¿La firma de Alice es válida? {firma_valida}")

# Ejemplo de un ataque: Eve intenta hacerse pasar por Alice
print("\n--- Simulación de ataque ---")
# Eve no tiene la clave privada de Alice, pero intenta enviar un mensaje a Bob
clave_privada_eve, clave_publica_eve = generar_par_claves_rsa()
mensaje_eve = "Hola Bob, soy Alice. Por favor, envíame dinero."
print(f"Eve intenta enviar: {mensaje_eve}")

# Eve firma con su propia clave privada
datos_cifrados_eve = enviar_mensaje_seguro(mensaje_eve, clave_privada_eve, clave_publica_bob)

# Bob recibe y verifica con la clave pública de Alice
mensaje_recibido_eve, firma_valida_eve = recibir_mensaje_seguro(datos_cifrados_eve, clave_privada_bob, clave_publica_alice)

print(f"Bob recibe: {mensaje_recibido_eve.decode()}")
print(f"¿La firma es válida? {firma_valida_eve}  <- Bob detecta que no es de Alice")


Alice quiere enviar: Hola Bob, este es un mensaje secreto de Alice.
Bob recibe: Hola Bob, este es un mensaje secreto de Alice.
¿La firma de Alice es válida? True

--- Simulación de ataque ---
Eve intenta enviar: Hola Bob, soy Alice. Por favor, envíame dinero.
Bob recibe: Hola Bob, soy Alice. Por favor, envíame dinero.
¿La firma es válida? False  <- Bob detecta que no es de Alice


## Conclusión

En este notebook, hemos explorado los conceptos fundamentales de la criptografía y su implementación práctica en Python. Hemos visto:

1. **Cifrado simétrico**: Utilizamos la misma clave para cifrar y descifrar (AES, César).
2. **Cifrado asimétrico**: Utilizamos un par de claves pública/privada (RSA).
3. **Funciones hash**: Convertimos datos en una huella digital de longitud fija (SHA-256, SHA-512).
4. **Firmas digitales**: Verificamos la autenticidad e integridad de los mensajes.
5. **Aplicaciones prácticas**: Cifrado de archivos y comunicación segura.

La criptografía es esencial para la seguridad de la información en el mundo digital actual. Estos conceptos y técnicas son la base de muchas aplicaciones y protocolos de seguridad que utilizamos a diario, como HTTPS, SSH, mensajería segura, etc.

### Recomendaciones de seguridad

- Nunca implementes tus propios algoritmos criptográficos para uso en producción. Utiliza bibliotecas bien establecidas y auditadas.
- Mantén tus claves privadas seguras y nunca las compartas.
- Utiliza contraseñas fuertes y únicas para cada servicio.
- Actualiza regularmente tus bibliotecas criptográficas para incorporar parches de seguridad.

### Recursos adicionales

- [Documentación de la biblioteca cryptography](https://cryptography.io/en/latest/)
- [PyCryptodome](https://pycryptodome.readthedocs.io/en/latest/)
- [Curso de Criptografía de Coursera](https://www.coursera.org/learn/crypto)
- [Libro: "Serious Cryptography" de Jean-Philippe Aumasson](https://nostarch.com/seriouscrypto)